In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains


email = "email" #Enter email
password = "password"  # Enter Password

# Path to chromedriver
path = r"C:\Users\risha\seleniumtwitterstock\chromedriver.exe"

service = Service(path)
driver = webdriver.Chrome(service=service)
driver.maximize_window()

def login_and_navigate_to_marketplace(driver, email, password):
    driver.get('https://www.facebook.com/login')
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'email'))).send_keys(email)
    driver.find_element(By.NAME, 'pass').send_keys(password)
    driver.find_element(By.NAME, 'pass').send_keys(Keys.RETURN)
    time.sleep(5) 

    driver.get('https://www.facebook.com/marketplace')
    time.sleep(5) 

def scroll_and_load_listings(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)  
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def send_offer_message(driver, item_url):
    try:
        driver.get(item_url)
        print(f"Visiting {item_url}")

        # Extract the price using CSS selector
        price_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x676frb.x1lkfr7t.x1lbecb7.x1s688f.xzsf02u'))
        )
        price_text = price_element.get_attribute("innerText")
        price = float(price_text.replace('$', '').replace(',', ''))
        offer_price = price / 1.2
        print(f"Price of the listing: {price}")

        offer_message = f'Hello, I am interested in your item. Would you accept ${offer_price:.2f}?'

        message_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//span[contains(@class, "x1lliihq") and text()="Message"]'))
        )
        driver.execute_script("arguments[0].click();", message_button)

        time.sleep(4)

        type_message_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//label[@aria-label="Please type your message to the seller"]'))
        )
        driver.execute_script("arguments[0].click();", type_message_element)

        time.sleep(1)

        message_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//label[@aria-label="Please type your message to the seller"]//textarea'))
        )
        ActionChains(driver).move_to_element(message_input).click().send_keys(offer_message).perform()

        send_message_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//span[contains(@class, "x1lliihq") and text()="Send message"]'))
        )
        driver.execute_script("arguments[0].click();", send_message_button)

        print(f"Sent message for {item_url}")

        time.sleep(2)  
    except TimeoutException:
        print(f"Timeout while trying to send message for {item_url}")
    except NoSuchElementException:
        print(f"Element not found while trying to send message for {item_url}")
    except Exception as e:
        print(f"An error occurred: {e}")

def main():
    try:
        login_and_navigate_to_marketplace(driver, email, password)
        scroll_and_load_listings(driver)

        listings = driver.find_elements(By.XPATH, '//a[contains(@href, "/marketplace/item/")]')
        listing_urls = [listing.get_attribute('href') for listing in listings]

        for url in listing_urls:
            send_offer_message(driver, url)
    except Exception as e:
        print(f"An error occurred in the main function: {e}")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

Visiting https://www.facebook.com/marketplace/item/1035741101546738/?ref=browse_tab&referral_code=marketplace_top_picks&referral_story_type=top_picks&__tn__=!%3AD
An error occurred: could not convert string to float: 'Seller information\nSeller details'
Visiting https://www.facebook.com/marketplace/item/1032721871727770/?ref=browse_tab&referral_code=marketplace_top_picks&referral_story_type=top_picks&__tn__=!%3AD
An error occurred: could not convert string to float: 'Seller information\nSeller details'
Visiting https://www.facebook.com/marketplace/item/1181895969602396/?ref=browse_tab&referral_code=marketplace_top_picks&referral_story_type=top_picks&__tn__=!%3AD
Price of the listing: 4455.0
Timeout while trying to send message for https://www.facebook.com/marketplace/item/1181895969602396/?ref=browse_tab&referral_code=marketplace_top_picks&referral_story_type=top_picks&__tn__=!%3AD
Visiting https://www.facebook.com/marketplace/item/1914495785666097/?ref=browse_tab&referral_code=marketp

: 